In [2]:
import numpy as np
from skimage import io
from helperfuncs import image_slice
from keras import models
from keras import layers
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

## Preprocessing of Data

In [12]:
directory = open('path_to_datafolder.txt','r').readlines()[0].strip('\n')

In [3]:
# Breaking up the 1024x1024 images into four 512x512
#break up images:
image_slice(directory,'image')
image_slice(directory,'label')

done!
done!


In [14]:
#creating training and testing sets
images = io.ImageCollection(directory+'/sliced_images/*.png')
label_masks = io.ImageCollection(directory + '/sliced_labels/*.png')

In [21]:
split = int(len(images)*0.75)
trainX = images[:split]
trainY = label_masks[:split]
testX = images[split:]
testY = label_masks[split:]

In [ ]:
# creating the data aumentation genreators for both the images and the label masks
data_gen_args = dict(featurewise_center=False,
                     featurewise_std_normalization=False,
                     rotation_range=90.,
                     rescale=1./255,
                     shear_range=0.2,
                     zoom_range=0.2,
                     horizontal_flip=True)

image_datagen = ImageDataGenerator(**data_gen_args)
mask_datagen = ImageDataGenerator(**data_gen_args)

seed = 1
image_datagen.fit(images, augment=True, seed=seed)
mask_datagen.fit(masks, augment=True, seed=seed)

image_generator = image_datagen.flow_from_directory(
    'data/images',
    class_mode=None,
    seed=seed)

mask_generator = mask_datagen.flow_from_directory(
    'data/masks',
    class_mode=None,
    seed=seed)

# combine generators into one which yields image and masks
train_generator = zip(image_generator, mask_generator)

## Making the Model

In [ ]:
#make the model

## Training the Model

In [ ]:
# train the model
model.fit_generator(
    train_generator,
    steps_per_epoch=2000,
    epochs=50)